In [1]:
import json
from pprint import pprint
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
import torch
import random

In [2]:
tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-Instruct-v0.2')
# model = AutoModelForCausalLM.from_pretrained(
#     'mistralai/Mistral-7B-Instruct-v0.2',
#     use_flash_attention_2 = True,
#     torch_dtype = torch.float16
# )
# tokenizer.pad_token = tokenizer.eos_token
# model.cuda()

### Audio

In [3]:
data = []

with open('mixtral-audio-instruction.jsonl') as fopen:
    for x in fopen:
        data.append(json.loads(x))

In [4]:
unique_conversations = []
conversation_set = set()

for data_ in data:

    conversations = data_['context']
    if conversations not in conversation_set:
        unique_conversations.append(data_)
        conversation_set.add(conversations)
        
len(unique_conversations)

59404

In [6]:
unique_conversations[0]['filename']

'output-audio/1-0-0.mp3'

In [10]:
batch_size = 100
prompt_list = []
selected_audio = set()

with open('multi-audio-chat-template.jsonl', 'w') as fopen:
    for x in tqdm(range(0, 100000, batch_size)):
        
        prompts = []
        audio0 = []
        audio1 = []
        for _ in range(batch_size):
            audio_ids = random.sample(list(set(range(len(unique_conversations))) - selected_audio), 2)
            selected_audio |= set(audio_ids)
            
            messages = [{'role': 'user',
                              'content': f"""
            Audio 1: {unique_conversations[audio_ids[0]]['context']} 
            Audio 2: {unique_conversations[audio_ids[1]]['context']}
            What is related between audio 1 and audio 2."""}]

            prompt = tokenizer.apply_chat_template(messages, tokenize = False)
            prompts.append(prompt)
            audio0.append((unique_conversations[audio_ids[0]]['context'],
                          unique_conversations[audio_ids[0]]['filename']))
            audio1.append((unique_conversations[audio_ids[1]]['context'], 
                           unique_conversations[audio_ids[1]]['filename']))
            
            
        d = {
            'prompts': prompts,
            'audio0': audio0,
            'audio1': audio1,
        }
        fopen.write(f'{json.dumps(d)}\n')